# Neural Networks

In this exercise you will learn how to implement a feedforward neural network and train it with backpropagation.

In [1]:
import numpy as np
from numpy.random import multivariate_normal
from numpy.random import uniform
from scipy.stats import zscore

We define two helper functions "init_toy_data" and "init_model" to create a simple data set to work on and a 2 layer neural network. 

First, we create toy data with categorical labels by sampling from different multivariate normal distributions for each class. 

In [2]:
def init_toy_data(num_samples,num_features, num_classes, seed=3):
    # num_samples: number of samples *per class*
    # num_features: number of features (excluding bias)
    # num_classes: number of class labels
    # seed: random seed
    np.random.seed(seed)
    X=np.zeros((num_samples*num_classes, num_features))
    y=np.zeros(num_samples*num_classes)
    for c in range(num_classes):
        # initialize multivariate normal distribution for this class:
        # choose a mean for each feature
        means = uniform(low=-10, high=10, size=num_features)
        # choose a variance for each feature
        var = uniform(low=1.0, high=5, size=num_features)
        # for simplicity, all features are uncorrelated (covariance between any two features is 0)
        cov = var * np.eye(num_features)
        # draw samples from normal distribution
        X[c*num_samples:c*num_samples+num_samples,:] = multivariate_normal(means, cov, size=num_samples)
        # set label
        y[c*num_samples:c*num_samples+num_samples] = c
    return X,y


In [3]:
def init_model(input_size,hidden_size,num_classes, seed=3):
    # input size: number of input features
    # hidden_size: number of units in the hidden layer
    # num_classes: number of class labels, i.e., number of output units
    np.random.seed(seed)
    model = {}
    # initialize weight matrices and biases randomly
    model['W1'] = uniform(low=-1, high=1, size=(input_size, hidden_size))
    model['b1'] = uniform(low=-1, high=1, size=hidden_size)
    model['W2'] = uniform(low=-1, high=1, size=(hidden_size, num_classes))
    model['b2'] = uniform(low=-1, high=1, size=num_classes)
    return model

In [4]:
# create toy data
X,y= init_toy_data(2,4,3) # 2 samples per class; 4 features, 3 classes
# Normalize data
X = zscore(X, axis=0)
print('X: ' + str(X))
print('y: ' + str(y))

X: [[ 0.39636145  1.09468144 -0.89360845  0.91815536]
 [ 0.94419323 -0.94027869  1.22268078  1.29597409]
 [-1.41577399  1.15477931 -0.62099631  0.08323307]
 [-1.35264614 -0.13598976 -1.14221784  0.26928935]
 [ 0.9352123   0.38225626  1.419864   -1.51152157]
 [ 0.49265316 -1.55544856  0.01427781 -1.0551303 ]]
y: [0. 0. 1. 1. 2. 2.]


We now initialise our neural net with one hidden layer consisting of $10$ units and and an output layer consisting of $3$ units. Here we expect (any number of) training samples with $4$ features. We do not apply any activation functions yet. The following figure shows a graphical representation of this neuronal net. 
<img src="nn.graphviz.png"  width="30%" height="30%">

In [5]:
# initialize model
model = init_model(input_size=4, hidden_size=10, num_classes=3)

print('model: ' + str(model))
print('model[\'W1\'].shape: ' + str(model['W1'].shape))
print('model[\'W2\'].shape: ' + str(model['W2'].shape))
print('model[\'b1\'].shape: ' + str(model['b1'].shape))
print('model[\'b12\'].shape: ' + str(model['b2'].shape))
print('number of parameters: ' + str((model['W1'].shape[0] * model['W1'].shape[1]) + 
     np.sum(model['W2'].shape[0] * model['W2'].shape[1]) + 
     np.sum(model['b1'].shape[0]) +
     np.sum(model['b2'].shape[0] )))

model: {'W1': array([[ 0.10159581,  0.41629565, -0.41819052,  0.02165521,  0.78589391,
         0.79258618, -0.74882938, -0.58551424, -0.89706559, -0.11838031],
       [-0.94024758, -0.08633355,  0.2982881 , -0.44302543,  0.3525098 ,
         0.18172563, -0.95203624,  0.11770818, -0.48149511, -0.16979761],
       [-0.43294984,  0.38627584, -0.11909256, -0.68626452,  0.08929804,
         0.56062953, -0.38727294, -0.55608423, -0.22405748,  0.8727673 ],
       [ 0.95199084,  0.34476735,  0.80566822,  0.69150174, -0.24401192,
        -0.81556598,  0.30682181,  0.11568152, -0.27687047, -0.54989099]]), 'b1': array([-0.18696017, -0.0621195 , -0.46152884, -0.41641445, -0.0846272 ,
        0.72106783,  0.17250581, -0.43302428, -0.44404499, -0.09075585]), 'W2': array([[-0.58917931, -0.59724258,  0.02807012],
       [-0.82554126, -0.03282894, -0.27564758],
       [ 0.41537324,  0.49349245,  0.38218584],
       [ 0.37836083, -0.25279975,  0.33626961],
       [-0.32030267,  0.14558774, -0.34838568]

<b>Exercise 1</b>: Implement softmax layer.

Implement the softmax function given by 

$softmax(x_i) = \frac{e^{x_i}}{{\sum_{j\in 1...J}e^{x_j}}}$, 

where $J$ is the total number of classes, i.e. the length of  **x** .

Note: Implement the function such that it takes a matrix X of shape (N, J) as input rather than a single instance **x**; N is the number of instances.

In [28]:
def softmax(X):
    
    #exponents = np.exp(X)
    #print(exponents)
    #sumofexp = np.sum(exponents[])    
    #print(sumofexp)
    #print(exponents/sumofexp)
    
    x = X - X.max(axis=1, keepdims=True)
    #print(x)
    y = np.exp(X)
    #print(y)
    #print(y / y.sum(axis=1, keepdims=True))
    return y / y.sum(axis=1, keepdims=True)
    #return exponents/sumofexp

Check if everything is correct.

In [29]:
x = np.array([[0.1, 0.7],[0.7,0.4]])
exact_softmax = np.array([[ 0.35434369,  0.64565631],
                         [ 0.57444252,  0.42555748]])
sm = softmax(x)

difference = np.sum(np.abs(exact_softmax - sm))
try:
    assert difference < 0.000001   
    print("Testing successful.")
except:
    print("Tests failed.")

Testing successful.


<b>Exercise 2</b>: Implement the forward propagation algorithm for the model defined above.

The activation function of the hidden neurons is a Rectified Linear Unit $relu(x)=max(0,x)$ (to be applied element-wise to the hidden units)
The activation function of the output layer is a softmax function as (as implemented in Exercise 1).

The function should return both the activation of the hidden units (after having applied the $relu$ activation function) (shape: $(N, num\_hidden)$) and the softmax model output (shape: $(N, num\_classes)$). 

In [40]:
def forward_prop(X,model):
    ###############################################
    # INSERT YOUR CODE HERE                       #
    ###############################################
    z1 = np.dot(X,model['W1']) + model['b1']
    #print(z1)
    #print(softmax(z1))
    h = np.maximum(0,softmax(z1))
    #print(h)
    z2 = np.dot(h,model['W2']) + model['b2']
    h2 = np.maximum(0,softmax(z2))
    print(h2)
    return z2,h2 # hidden_activations,probs

In [42]:
acts,probs = forward_prop(X, model)
correct_probs = np.array([[0.22836388, 0.51816433, 0.25347179],
                            [0.15853289, 0.33057078, 0.51089632],
                            [0.40710319, 0.41765056, 0.17524624],
                            [0.85151353, 0.03656425, 0.11192222],
                            [0.66016592, 0.19839791, 0.14143618],
                            [0.70362036, 0.08667923, 0.20970041]])

# the difference should be very small.
difference =  np.sum(np.abs(probs - correct_probs))
#print(probs.shape)
#print((X.shape[0],len(set(y))))
try:
    assert probs.shape==(X.shape[0],len(set(y)))
    assert difference < 0.00001   
    print("Testing successful.")
except:
    print("Tests failed.")

[[0.24004851 0.49964514 0.26030635]
 [0.22490488 0.48385872 0.29123641]
 [0.29046064 0.4726512  0.23688816]
 [0.33933928 0.39294182 0.2677189 ]
 [0.31459734 0.42908102 0.25632163]
 [0.31123832 0.41970479 0.26905689]]
(6, 3)
(6, 3)
Tests failed.


<b>Exercise 3:</b> How would you train the above defined neural network? Which loss-function would you use? You do not need to implement this.

# Part 2 (Neural Net using Keras)

Instead of implementing the model learning ourselves, we can use the neural network library Keras for Python (https://keras.io/). Keras is an abstraction layer that either builds on top of Theano or Google's Tensorflow. So please install Keras and Tensorflow/Theano for this lab.

<b>Exercise 4:</b>
    Implement the same model as above using Keras:
    
    ** 1 hidden layer à 10 units
    ** softmax output layer à three units
    ** 4 input features
    
Compile the model using categorical cross-entropy (also referred to as 'softmax-loss') as loss function and using categorical crossentropy together categorical accuracy as metrics for runtime evaluation during training.

In [57]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.layers.core import Activation
from keras.losses import categorical_crossentropy

# define the model 
# Define Sequential model with 3 layers
model = Sequential(
    [
        Dense(10, input_dim=4,activation="relu", name="layer1"),
               Dense(3, name="layer3"),
    ]
)
opt = Adam(learning_rate=1e-3)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

The description of the current network can always be looked at via the summary method. The layers can be accessed via model.layers and weights can be obtained with the method get_weights. Check if your model is as expected. 

In [58]:
# Check model architecture and initial weights.

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 10)                50        
_________________________________________________________________
layer3 (Dense)               (None, 3)                 33        
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________


<b>Exercise 5:</b> Train the model on the toy data set generated below: 

Hints: 

* Keras expects one-hot-coded labels 

* Don't forget to normalize the data

In [59]:
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.layers.experimental.preprocessing import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_string_categorical_feature(feature, name, dataset):
    # Create a StringLookup layer which will turn strings into integer indices
    index = StringLookup()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    index.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = index(feature)

    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a dataset of indices
    feature_ds = feature_ds.map(index)

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(encoded_feature)
    return encoded_feature


def encode_integer_categorical_feature(feature, name, dataset):
    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(feature)
    return encoded_feature

ImportError: cannot import name 'CategoryEncoding' from 'tensorflow.keras.layers.experimental.preprocessing' (C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\keras\api\_v2\keras\layers\experimental\preprocessing\__init__.py)

In [ ]:
X, y = init_toy_data(1000,4,3, seed=3)

#https://keras.io/examples/structured_data/structured_data_classification_from_scratch/